In [30]:
import cv2
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import numpy as np
from scipy.signal import butter, lfilter, find_peaks
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.fft import fft
from tensorflow.keras.models import load_model

In [31]:
foreheads = load_model('forehead.h5')

# Heart-Rate measurement 

In [44]:


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)
foreheads = load_model('forehead.h5')

filter_order = 4
low_cutoff = 0.7
high_cutoff = 4.0

buffer_size = 100
color_buffer = np.zeros(buffer_size)
sample_rate = 30.0
time_vector = np.arange(buffer_size) / sample_rate
update = 0
total = 0
size = 30
h , w = size,size
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))
    
    yhat = foreheads.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    if yhat[0] == 1:
        x1 = sample_coords[0] * 640 
        y1 = sample_coords[1] * 480
        x2 = sample_coords[2] * 640
        y2 = sample_coords[3] * 480
        
        x = int((x1 + x2) / 2)
        y = int((y1 + y2) / 2)
        
        
        colors = frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2),]
        cv2.rectangle(frame, (x-w//2, y-h//2), (x+w//2, y+h//2), (0, 255, 0), 2)
        roi = (x, y, size, size) 

        roi_frame = frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2)]
        roi_gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
        b, a = butter(filter_order, [low_cutoff/sample_rate*2, high_cutoff/sample_rate*2], btype='bandpass')
        roi_filtered = lfilter(b, a, roi_gray)
        

        color_buffer = np.roll(color_buffer, -1)
        color_buffer[-1] = np.mean(roi_filtered)
        signal = color_buffer - np.mean(color_buffer)
        peaks, _ = find_peaks(signal, height=0)
        if len(peaks) >= 2:
            bpm = 70.0 / np.mean(np.diff(peaks)) * (buffer_size / sample_rate) 
            total += bpm
        else:
            bpm = 0.0
        
        
        if (update%20)==0:
            val = total/20
            total = 0

        frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2),:] = np.zeros((size,size,3))
    else:
        val = 0
    
    cv2.putText(frame, 'Heart rate: {:.1f} bpm'.format(val), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    update+=1

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 89ms/step


In [37]:
import cv2
import numpy as np
from scipy.fft import fft

# Open the default camera (front-facing camera)
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error opening camera")
    exit()

# Define the foreheads bounding box
x, y, h, w = 350, 150, 100, 100
real_part = 60
arr = []
val = 60
# Loop over the frames from the camera
while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Check if the frame is read successfully
    if not ret:
        print("Error reading frame")
        break

    # Crop the foreheads region
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))
    
    yhat = foreheads.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] >= 0.8: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [640,480]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [640,480]).astype(int)), 
                            (0,0,0), 2)

        x1 = sample_coords[0] * 640 
        y1 = sample_coords[1] * 480
        x2 = sample_coords[2] * 640
        y2 = sample_coords[3] * 480
        
        x = int((x1 + x2) / 2)
        y = int((y1 + y2) / 2)

        forehead = frame[y:y+50, x:x+50,:]

        # Convert the foreheads pixels to HSV
        forehead_hsv = cv2.cvtColor(forehead, cv2.COLOR_BGR2HSV)
        hue = np.array([])
        for i in range(100):
            for j in range(100):
                #print(forehead_hsv[i][j][0])
                if forehead_hsv[i][j][0] >= 0 and forehead_hsv[i][j][0] <= 0.9*360:
                    hue = np.append(hue,forehead_hsv[i][j][0]/360)
        arr.append(np.mean(hue))
        
        if len(arr) % 10 ==0:
            time_arr = arr[-10:]
            freq_arr = fft(time_arr)
            temp = []
            for i in freq_arr:
                if i >= 0.8 and i <= 2:
                    temp.append(i)
            if len(temp)==0:
                val = 0
            else: 
                val = max(temp)
            #val = max(i for i in freq_arr if i>=0.8 and i<=2)
            heart_rate = val *60
            #x = (49.976749999999996+0j)
            real_part = heart_rate.real
            print(real_part)

            print("heart rate: ",real_part)


        # Draw the bounding box and display the average FP value on the frame
        cv2.rectangle(frame, (x-w//2, y-h//2), (x+w//2, y+h//2), (0, 255, 0), 2)
        cv2.putText(frame, f"Heart Rate: {real_part}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('frame', frame)

    # Wait for the user to press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()

Error reading frame


# Heart-Rate measurement in lowLight

In [35]:
import cv2
import numpy as np
from scipy.signal import butter, lfilter, find_peaks
import tensorflow as tf
from tensorflow.keras.models import load_model

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)
foreheads = load_model('forehead.h5')

filter_order = 4
low_cutoff = 0.7
high_cutoff = 4.0

buffer_size = 100
color_buffer = np.zeros(buffer_size)

sample_rate = 30.0
time_vector = np.arange(buffer_size) / sample_rate
update = 0
total = 0
size = 30
h , w = size,size
while True:
    ret, frame = cap.read()
    if not ret:
        break
    

    r = frame[:,:,2]
    g = frame[:,:,1]
    b = frame[:,:,0]
    equalizedr =  cv2.equalizeHist(r)
    equalizedg =  cv2.equalizeHist(g)
    equalizedb =  cv2.equalizeHist(b)
    frame = cv2.merge([equalizedb,equalizedg,equalizedr])
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb,(120, 120))
    
    yhat = foreheads.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]

    x1 = sample_coords[0] * 640 
    y1 = sample_coords[1] * 480
    x2 = sample_coords[2] * 640
    y2 = sample_coords[3] * 480
    
    x = int((x1 + x2) / 2)
    y = int((y1 + y2) / 2)
    
    
    colors = frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2),]
    cv2.rectangle(frame, (x-w//2, y-h//2), (x+w//2, y+h//2), (0, 255, 0), 2)
    roi = (x, y, size, size) 

    roi_frame = frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2)]
    roi_gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
    b, a = butter(filter_order, [low_cutoff/sample_rate*2, high_cutoff/sample_rate*2], btype='bandpass')
    roi_filtered = lfilter(b, a, roi_gray)
    
    color_buffer = np.roll(color_buffer, -1)
    color_buffer[-1] = np.mean(roi_filtered)
    
    signal = color_buffer - np.mean(color_buffer)
    peaks, _ = find_peaks(signal, height=0)
    if len(peaks) >= 2:
        bpm = 70.0 / np.mean(np.diff(peaks)) * (buffer_size / sample_rate) 
        total += bpm
    else:
        bpm = 0.0
    
    
    if (update%20)==0:
        val = total/20
        total = 0

    frame[y-int(size/2):y+int(size/2),x-int(size/2):x+int(size/2),:] = np.zeros((size,size,3))

    cv2.putText(frame, 'Heart rate: {:.1f} bpm'.format(val), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    update+=1
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 139ms/step
